In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'loan-status-prediction-with-added-nans:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4589662%2F7831548%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240324%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240324T132037Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3a0809b6cc746598822b45eb6edd12a8ef33811e2d8e9a74b5cf8c2ec82926ccabe64d51fbb2dd3d96e254aa2c42b4cca1c2f5c1dbacb7aaf891e8a9e430d891e7c246ff05f85db78acf127614ba627330138705d2c9e277d8563eac7adf589a8527e79a7863faabbd7389191d0f3a44644faabb93ab9eefda93ec3f292084eb56bb024814412ce4113aed96710e9fd6d205d7d7761e91b6c07cbebae48d448abc66919d68004d5c7ce9fb230f4b6ae6c7ae0bc219aeb185f172c6a58c657c78a14ea0b2958cd1424d36aa85e835e012363df823d8d11114d9fd3148922eb3c0b6ffa2f245230eff0983f664b21fbb9baff0f1037e0896deac6ecd4ac172f004'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 6235 bytes downloaded
Downloaded and uncompressed: loan-status-prediction-with-added-nans
Data source import complete.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/loan-status-prediction-with-added-nans/loan_data_1.csv


In [ ]:
data_file_path = "/kaggle/input/loan-status-prediction-with-added-nans/loan_data_1.csv"
dataset = pd.read_csv(data_file_path)
dataset.head()

,Unnamed: 0,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,0,LP001003,Male,Yes,1,Graduate,No,4583.0,1508.0,128.0,360.0,1.0,Rural,N
1,1,LP001005,Male,Yes,0,Graduate,Yes,3000.0,0.0,66.0,360.0,1.0,Urban,Y
2,2,LP001006,Male,Yes,0,Not Graduate,No,2583.0,2358.0,120.0,360.0,1.0,Urban,Y
3,3,LP001008,Male,No,0,Graduate,No,6000.0,0.0,141.0,360.0,1.0,Urban,Y
4,4,LP001013,Male,Yes,0,Not Graduate,No,2333.0,1516.0,95.0,360.0,1.0,Urban,Y


In [ ]:
# Preprocess dataset
# Output
dataset["Loan_Status"] = dataset["Loan_Status"].map(lambda status: 1 if status == "Y" else 0)
# Gender
dataset["Male"] = (dataset["Gender"] == "Male").astype(int)
dataset["Female"] = (dataset["Gender"] == "Female").astype(int)
# Marriage
dataset["Married"] = dataset["Married"].map(lambda status: 1 if status == "Yes" else 0)
# Dependents
dataset["Dependents"] = dataset["Dependents"].fillna("Unknown")
value_counts = dataset["Dependents"].value_counts()
dataset["Dependents"] = dataset["Dependents"].map(lambda d: value_counts[d])
# Education
dataset["Graduate"] = (dataset["Education"] == "Graduate").astype(int)
# Self-employed
dataset["Self_Employed"] = (dataset["Self_Employed"] == "Yes").astype(int)
# Income
dataset["ApplicantIncome"] = dataset["ApplicantIncome"].fillna(dataset["ApplicantIncome"].mean())
dataset["CoapplicantIncome"] = dataset["CoapplicantIncome"].fillna(dataset["CoapplicantIncome"].mean())
# Loan
dataset["LoanAmount"] = dataset["LoanAmount"].fillna(dataset["LoanAmount"].mean())
# Credit History
dataset["Credit_History"] = dataset["Credit_History"].astype(str)
value_counts = dataset["Credit_History"].value_counts()
dataset["Credit_History"] = dataset["Credit_History"].map(lambda h: value_counts[h])
# Property Area
value_counts = dataset["Property_Area"].value_counts()
dataset["Property_Area"] = dataset["Property_Area"].map(lambda a: value_counts[a])

In [ ]:
records = dataset.loc[dataset["Male"] == 1]
approval_ratio = records["Loan_Status"].sum() / records["Loan_Status"].count()
print("Male Approval Percentage:", approval_ratio * 100.0)

records = dataset.loc[dataset["Female"] == 1]
approval_ratio = records["Loan_Status"].sum() / records["Loan_Status"].count()
print("Female Approval Percentage:", approval_ratio * 100.0)

Male Approval Percentage: 72.5085910652921
Female Approval Percentage: 69.41176470588235


In [ ]:
records = dataset.loc[dataset["Married"] == 1]
approval_ratio = records["Loan_Status"].sum() / records["Loan_Status"].count()
print("Married Approval Percentage:", approval_ratio * 100.0)

records = dataset.loc[dataset["Married"] == 0]
approval_ratio = records["Loan_Status"].sum() / records["Loan_Status"].count()
print("Unmarried Approval Percentage:", approval_ratio * 100.0)

Married Approval Percentage: 74.56140350877193
Unmarried Approval Percentage: 66.01307189542483


In [ ]:
records = dataset.loc[dataset["Graduate"] == 1]
approval_ratio = records["Loan_Status"].sum() / records["Loan_Status"].count()
print("Graduate Approval Percentage:", approval_ratio * 100.0)

records = dataset.loc[dataset["Graduate"] == 0]
approval_ratio = records["Loan_Status"].sum() / records["Loan_Status"].count()
print("Non-Graduate Approval Percentage:", approval_ratio * 100.0)

Graduate Approval Percentage: 73.35766423357664
Non-Graduate Approval Percentage: 65.42056074766354


In [ ]:
records = dataset.loc[dataset["Self_Employed"] == 1]
approval_ratio = records["Loan_Status"].sum() / records["Loan_Status"].count()
print("Self-Employed Approval Percentage:", approval_ratio * 100.0)

records = dataset.loc[dataset["Self_Employed"] == 0]
approval_ratio = records["Loan_Status"].sum() / records["Loan_Status"].count()
print("Non-Self-Employed Approval Percentage:", approval_ratio * 100.0)

Self-Employed Approval Percentage: 68.57142857142857
Non-Self-Employed Approval Percentage: 71.38728323699422


In [ ]:
train_labels = ["Male", "Female", "Married", "Graduate", "ApplicantIncome", "CoapplicantIncome", "LoanAmount", "Credit_History", "Property_Area"]
output_label = "Loan_Status"
dataset = dataset[[*train_labels, output_label]]

In [ ]:
import tensorflow as tf

In [ ]:
BATCH_SIZE = 32
DATASET_SIZE = dataset.shape[0]
train_size = int(0.7 * DATASET_SIZE)
val_size = int(0.15 * DATASET_SIZE)

# Shuffle the dataset
dataset = dataset.sample(frac=1)
train_df = dataset[train_labels]
output_df = dataset[output_label]

# Convert the data to tensors
x_train = tf.convert_to_tensor(train_df.iloc[:train_size])
y_train = tf.convert_to_tensor(output_df.iloc[:train_size])

x_val = tf.convert_to_tensor(train_df.iloc[train_size:train_size + val_size])
y_val = tf.convert_to_tensor(output_df.iloc[train_size:train_size + val_size])

x_test = tf.convert_to_tensor(train_df.iloc[train_size + val_size:])
y_test = tf.convert_to_tensor(output_df.iloc[train_size + val_size:])

normalizer = tf.keras.layers.Normalization()
normalizer.adapt(x_train)

In [ ]:
def build_model():
    model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(16),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(8),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])
    return model

In [ ]:
model = build_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 normalization (Normalizati  (None, 9)                 19        
 on)                                                             
                                                                 
 dense (Dense)               (None, 16)                160       
                                                                 
 batch_normalization (Batch  (None, 16)                64        
 Normalization)                                                  
                                                                 
 activation (Activation)     (None, 16)                0         
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 8)                 1

In [ ]:
model.fit(x_train, y_train, epochs=200, batch_size=BATCH_SIZE, validation_data=(x_val, y_val))

Epoch 1/200
9/9 [==============================] - 4s 71ms/step - loss: 0.7983 - accuracy: 0.5827 - val_loss: 0.5933 - val_accuracy: 0.7368
Epoch 2/200
9/9 [==============================] - 0s 13ms/step - loss: 0.7859 - accuracy: 0.6316 - val_loss: 0.5867 - val_accuracy: 0.7368
Epoch 3/200
9/9 [==============================] - 0s 11ms/step - loss: 0.7704 - accuracy: 0.6015 - val_loss: 0.5773 - val_accuracy: 0.7544
Epoch 4/200
9/9 [==============================] - 0s 8ms/step - loss: 0.7131 - accuracy: 0.6541 - val_loss: 0.5685 - val_accuracy: 0.7719
Epoch 5/200
9/9 [==============================] - 0s 9ms/step - loss: 0.6769 - accuracy: 0.6805 - val_loss: 0.5603 - val_accuracy: 0.7719
Epoch 6/200
9/9 [==============================] - 0s 10ms/step - loss: 0.7184 - accuracy: 0.6278 - val_loss: 0.5525 - val_accuracy: 0.7895
Epoch 7/200
9/9 [==============================] - 0s 8ms/step - loss: 0.6902 - accuracy: 0.6729 - val_loss: 0.5448 - val_accuracy: 0.7895
Epoch 8/200
9/9 [======

In [ ]:
model.evaluate(x_test, y_test)

2/2 [==============================] - 0s 7ms/step - loss: 0.5098 - accuracy: 0.8276


[0.5097941160202026, 0.8275862336158752]